In [2]:
import os
from pathlib import Path

import pandas as pd

from draft_optimizer.src.platform.espn import League
from draft_optimizer.src.utils import DATA_DIR

In [3]:
# Specify info
league_id = 88497130
years = [2019, 2020, 2021, 2022]

In [4]:
# Loop over years
for year in years:
    # Get league
    league = League(id=league_id, year=year)

    # Get pro teams and schedule
    pro_teams, pro_schedule = league.get_pro_schedule()

    # Get players
    # max_players = 25
    max_players = None
    players = league.get_players(max_players=max_players)

    # Convert to df
    pro_teams_df = (
        pd.DataFrame.from_dict([v.dict() for v in pro_teams.values()]).sort_values("id").reset_index(drop=True)
    )
    pro_schedule_df = (
        pd.DataFrame.from_dict([v2.dict() for v1 in pro_schedule.values() for v2 in v1])
        .sort_values(["week", "date", "home_id", "away_id"])
        .reset_index(drop=True)
    )
    players_df = pd.DataFrame.from_dict([v.dict() for v in players.values()]).sort_values("id").reset_index(drop=True)

    # Export data
    league_dir = os.path.join(DATA_DIR, f"espn_{league_id}", str(year))
    if not os.path.isdir(league_dir):
        Path(league_dir).mkdir(parents=True, exist_ok=True)
    pro_teams_df.to_csv(os.path.join(league_dir, "pro_teams.csv"), index=False)
    pro_schedule_df.to_csv(os.path.join(league_dir, "pro_schedule.csv"), index=False)
    players_df.to_csv(os.path.join(league_dir, "pro_players.csv"), index=False)